<a href="https://colab.research.google.com/github/polayao/MetNumUN2023I/blob/main/Lab7/Week4IterativeMethodsForLinearSystemsGroup1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Grupo 1

Obed Felipe Espinosa Angarita

Karen Tatiana Alvarez Baez

Paula Liliana Olaya Oyola

# Simple iteration for systems of linear equations

First, generate a random diagonally dominant matrix, for testing.

In [ ]:
import numpy as np
rndm = np.random.RandomState(1234)

n = 10
A = rndm.uniform(size=(n, n)) + np.diagflat([15]*n)
b = rndm.uniform(size=n)

In [ ]:
A

array([[1.51915195e+01, 6.22108771e-01, 4.37727739e-01, 7.85358584e-01,
        7.79975808e-01, 2.72592605e-01, 2.76464255e-01, 8.01872178e-01,
        9.58139354e-01, 8.75932635e-01],
       [3.57817270e-01, 1.55009951e+01, 6.83462935e-01, 7.12702027e-01,
        3.70250755e-01, 5.61196186e-01, 5.03083165e-01, 1.37684496e-02,
        7.72826622e-01, 8.82641191e-01],
       [3.64885984e-01, 6.15396178e-01, 1.50753812e+01, 3.68824006e-01,
        9.33140102e-01, 6.51378143e-01, 3.97202578e-01, 7.88730143e-01,
        3.16836122e-01, 5.68098653e-01],
       [8.69127390e-01, 4.36173424e-01, 8.02147642e-01, 1.51437668e+01,
        7.04260971e-01, 7.04581308e-01, 2.18792106e-01, 9.24867629e-01,
        4.42140755e-01, 9.09315959e-01],
       [5.98092228e-02, 1.84287084e-01, 4.73552788e-02, 6.74880944e-01,
        1.55946248e+01, 5.33310163e-01, 4.33240627e-02, 5.61433080e-01,
        3.29668446e-01, 5.02966833e-01],
       [1.11894318e-01, 6.07193706e-01, 5.65944643e-01, 6.76406199e-03,
   

# I.  Jacobi iteration

Given

$$
A x = b
$$

separate the diagonal part $D$,

$$ A = D + (A - D) $$

and write

$$
x = D^{-1} (D - A) x + D^{-1} b\;.
$$

Then iterate

$$
x_{n + 1} = B x_{n} + c\;,
$$

where 

$$
B = D^{-1} (A - D) \qquad \text{and} \qquad c = D^{-1} b
$$


Let's construct the matrix and the r.h.s. for the Jacobi iteration

In [ ]:
diag_1d = np.diag(A)

B = -A.copy()
np.fill_diagonal(B, 0)

D = np.diag(diag_1d)
invD = np.diag(1./diag_1d)
BB = invD @ B 
c = invD @ b

In [ ]:
# sanity checks
from numpy.testing import assert_allclose

assert_allclose(-B + D, A)


# xx is a "ground truth" solution, compute it using a direct method
xx = np.linalg.solve(A, b)

np.testing.assert_allclose(A@xx, b)
np.testing.assert_allclose(D@xx, B@xx + b)
np.testing.assert_allclose(xx, BB@xx + c)

Check that $\| B\| \leqslant 1$:

In [ ]:
np.linalg.norm(BB)

0.36436161983015336

### Do the Jacobi iteration

In [ ]:
n_iter = 50

x0 = np.ones(n)
x = x0
for _ in range(n_iter):
    x = BB @ x + c

In [ ]:
# Check the result:

A @ x - b

array([ 1.11022302e-16,  0.00000000e+00, -2.22044605e-16, -1.11022302e-16,
        1.11022302e-16,  0.00000000e+00, -2.42861287e-17,  0.00000000e+00,
       -2.77555756e-17,  1.11022302e-16])

### Task I.1

Collect the proof-of-concept above into a single function implementing the Jacobi iteration. This function should receive the r.h.s. matrix $A$, the l.h.s. vector `b`, and the number of iterations to perform.


The matrix $A$ in the illustration above is strongly diagonally dominant, by construction. 
What happens if the diagonal matrix elements of $A$ are made smaller? Check the convergence of the Jacobi iteration, and check the value of the norm of $B$.

(20% of the total grade)


In [ ]:
# ... ENTER YOUR CODE HERE ...

import numpy as np

def jacobi_iteration(M_A, V_b, num_iterations):
    n = M_A.shape[0]
    x = np.zeros(n)  # Initial guess for solution vector

    norms_diff = []
    norms_B = []

    for _ in range(num_iterations):
        x_new = np.zeros(n)
        for i in range(n):
            x_new[i] = (V_b[i] - np.dot(M_A[i, :i], x[:i]) - np.dot(M_A[i, i+1:], x[i+1:])) / M_A[i, i]
        norms_diff.append(np.linalg.norm(x_new - x))
        norms_B.append(np.linalg.norm(M_A - np.diag(np.diagonal(M_A))))

        x = x_new

    return x, norms_diff, norms_B


# Example
A = np.array([[4, 1, 1],
              [1, 5, 2],
              [1, 2, 6]])

b = np.array([1, 2, 3])

num_iterations = 10

# Original matrix A
x_orig, diff_orig, norm_B_orig = jacobi_iteration(A, b, num_iterations)

# Modified matrix A with smaller diagonal elements
A_modified = np.array([[0.4, 0.1, 0.1],
                       [0.1, 0.5, 0.2],
                       [0.1, 0.2, 0.6]])

x_modified, diff_modified, norm_B_modified = jacobi_iteration(A_modified, b, num_iterations)

print("Original A:")
print("Solution vector:", x_orig)
print("Norms of differences between consecutive solutions:", diff_orig)
print("Norms of matrix B:", norm_B_orig)
print()

print("Modified A with smaller diagonal elements:")
print("Solution vector:", x_modified)
print("Norms of differences between consecutive solutions:", diff_modified)
print("Norms of matrix B:", norm_B_modified)

Original A:
Solution vector: [0.09229869 0.21595253 0.41187576]
Norms of differences between consecutive solutions: [0.687386354243376, 0.3791437722025775, 0.19777451035066282, 0.10704482121989833, 0.05698190978314714, 0.03064512627199415, 0.01638126971629196, 0.00879260153435083, 0.004706802221473611, 0.002524255646749693]
Norms of matrix B: [3.4641016151377544, 3.4641016151377544, 3.4641016151377544, 3.4641016151377544, 3.4641016151377544, 3.4641016151377544, 3.4641016151377544, 3.4641016151377544, 3.4641016151377544, 3.4641016151377544]

Modified A with smaller diagonal elements:
Solution vector: [0.92298693 2.15952527 4.11875764]
Norms of differences between consecutive solutions: [6.87386354243376, 3.7914377220257753, 1.9777451035066285, 1.0704482121989833, 0.5698190978314713, 0.3064512627199413, 0.16381269716291957, 0.08792601534350843, 0.04706802221473645, 0.025242556467497023]
Norms of matrix B: [0.3464101615137755, 0.3464101615137755, 0.3464101615137755, 0.3464101615137755, 0.

# II. Seidel's iteration.

##### Task II.1

Implement the Seidel's iteration. 

Test it on a random matrix. Study the convergence of iterations, relate to the norm of the iteration matrix.

(30% of the total grade)

In [ ]:
# ... ENTER YOUR CODE HERE ...
import numpy as np

def gauss_seidel_iteration(A, b, num_iterations):
    n = A.shape[0]
    x = np.zeros(n)  # Initial guess for solution vector

    norms_diff = []
    norms_B = []

    for _ in range(num_iterations):
        x_new = np.zeros(n)
        for i in range(n):
            x_new[i] = (b[i] - np.dot(A[i, :i], x_new[:i]) - np.dot(A[i, i+1:], x[i+1:])) / A[i, i]
        norms_diff.append(np.linalg.norm(x_new - x))
        norms_B.append(np.linalg.norm(A - np.tril(A) @ np.triu(A, 1)))

        x = x_new

    return x, norms_diff, norms_B

# Example
n = 5  # Size of the matrix
A = np.random.rand(n, n)  # Random matrix
b = np.random.rand(n)  # Random right-hand side vector
num_iterations = 10

x, diff, norm_B = gauss_seidel_iteration(A, b, num_iterations)

print("Solution vector:", x)
print("Norms of differences between consecutive solutions:", diff)
print("Norms of iteration matrix B:", norm_B)

Solution vector: [ 3962.12399076 -2247.30195851  -580.21134662    46.26534349
  -676.48455552]
Norms of differences between consecutive solutions: [2.9977191618735075, 4.19861576243853, 9.064445302908043, 19.738578817663214, 44.38717960667772, 100.22547793299411, 225.97456679871544, 509.00876409017945, 1146.1908121145125, 2580.819548152575]
Norms of iteration matrix B: [1.482023289047244, 1.482023289047244, 1.482023289047244, 1.482023289047244, 1.482023289047244, 1.482023289047244, 1.482023289047244, 1.482023289047244, 1.482023289047244, 1.482023289047244]


# III. Minimum residual scheme

### Task III.1

Implement the $\textit{minimum residual}$ scheme: an explicit non-stationary method, where at each step you select the iteration parameter $\tau_n$ to minimize the residual $\mathbf{r}_{n+1}$ given $\mathbf{r}_n$. Test it on a random matrix, study the convergence to the solution, in terms of the norm of the residual and the deviation from the ground truth solution (which you can obtain using a direct method). Study how the iteration parameter $\tau_n$ changes as iterations progress.

(50% of the grade)

In [ ]:
# ... ENTER YOUR CODE HERE ...
import numpy as np

def minres_scheme(A, b, num_iterations):
    n = A.shape[0]
    x = np.zeros(n)  # Initial guess for solution vector
    r = b - A @ x  # Initial residual vector
    p = r.copy()  # Initial search direction vector

    norms_resid = []
    norms_deviation = []
    tau_values = []

    for k in range(num_iterations):
        Ap = A @ p
        denom = np.dot(p, Ap)
        if denom == 0:
            break
        tau = np.dot(r, Ap) / denom
        x_new = x + tau * p
        r_new = r - tau * Ap

        norms_resid.append(np.linalg.norm(r_new))
        norms_deviation.append(np.linalg.norm(x_new - x))
        tau_values.append(tau)

        x = x_new
        r = r_new
        beta = np.linalg.norm(r)
        p = r + (beta / np.linalg.norm(r)) * p

    return x, norms_resid, norms_deviation, tau_values

# Example

n = 5  # Size of the matrix
A = np.random.rand(n, n)  # Random matrix
b = np.random.rand(n)  # Random right-hand side vector
num_iterations = 10

# Obtain ground truth solution using a direct method
x_ground_truth = np.linalg.solve(A, b)

x_minres, norms_resid, norms_deviation, tau_values = minres_scheme(A, b, num_iterations)

print("Ground truth solution:", x_ground_truth)
print("MINRES solution:", x_minres)
print("Norms of residual:", norms_resid)
print("Norms of deviation from ground truth solution:", norms_deviation)
print("Iteration parameter tau values:", tau_values)

Ground truth solution: [-2.78949331  1.5457915   0.23944318 -0.35412699  1.55915107]
MINRES solution: [  33.1777601   -63.41210021   33.2748841   148.05828644 -141.05040495]
Norms of residual: [1.9188564554418737, 4.632708784677799, 4.831248769707328, 9.196702577576312, 13.716102574181395, 21.944416737919965, 34.79196005047926, 53.20193651675264, 79.47066687789459, 116.39240738065561]
Norms of deviation from ground truth solution: [1.1905315163575663, 14.977257740702182, 4.193212704527434, 7.4661256694975044, 8.176023340920247, 13.982306668641717, 22.964271341074785, 35.86057428492157, 54.82100654298051, 81.89353592957342]
Iteration parameter tau values: [1.0, -6.983032012885541, 0.7330882216687573, 0.800636962700809, 0.47577352309886983, 0.4580620743324047, 0.4417946772768998, 0.4161972520170877, 0.3952477384829, 0.37686526210212495]
